In [1]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pprint
import pyspark
import pyspark.sql.functions as F

from pyspark.sql.functions import col
from pyspark.sql.types import StringType, IntegerType, FloatType, DateType, TimestampType

import utils.data_processing_bronze_table

import utils.data_processing_silver_table
import utils.data_processing_gold_table


## set up pyspark session

In [2]:
# Initialize SparkSession
spark = pyspark.sql.SparkSession.builder \
    .appName("dev") \
    .master("local[*]") \
    .getOrCreate()

# Set log level to ERROR to hide warnings
spark.sparkContext.setLogLevel("ERROR")
processor = utils.data_processing_silver_table.DataProcessor(spark)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/15 13:02:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## set up config

In [3]:
# set up config
snapshot_date_str = "2023-01-01"
start_date_str = "2023-01-01"
end_date_str = "2024-12-01"

In [4]:
# generate list of dates to process
def generate_first_of_month_dates(start_date_str, end_date_str):
    # Convert the date strings to datetime objects
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
    
    # List to store the first of month dates
    first_of_month_dates = []

    # Start from the first of the month of the start_date
    current_date = datetime(start_date.year, start_date.month, 1)

    while current_date <= end_date:
        # Append the date in yyyy-mm-dd format
        first_of_month_dates.append(current_date.strftime("%Y-%m-%d"))
        
        # Move to the first of the next month
        if current_date.month == 12:
            current_date = datetime(current_date.year + 1, 1, 1)
        else:
            current_date = datetime(current_date.year, current_date.month + 1, 1)

    return first_of_month_dates

dates_str_lst = generate_first_of_month_dates(start_date_str, end_date_str)
# dates_str_lst

## Build Bronze Table

In [5]:
# data_directory = 'data'
# list_of_raw_files = utils.data_processing_bronze_table.get_csv_files_in_directory(data_directory)
# for file_name in list_of_raw_files:
#     base_name = os.path.splitext(file_name)[0]
#     directory_path = f"datamart/bronze/{base_name}/"
#     csv_file_path = os.path.join(data_directory, file_name)
#     utils.data_processing_bronze_table.process_bronze_datalake(
#         directory_path=directory_path,
#         dates_list=dates_str_lst,
#         processing_function=utils.data_processing_bronze_table.process_bronze_table,
#         spark=spark,
#         csv_file_path=csv_file_path
#     )

Directory already exists: datamart/bronze/lms_loan_daily/
Processing date: 2023-01-01


Saved data to Parquet table at: datamart/bronze/lms_loan_daily/date=2023-01-01
Processing date: 2023-02-01
Saved data to Parquet table at: datamart/bronze/lms_loan_daily/date=2023-02-01
Processing date: 2023-03-01
Saved data to Parquet table at: datamart/bronze/lms_loan_daily/date=2023-03-01
Processing date: 2023-04-01
Saved data to Parquet table at: datamart/bronze/lms_loan_daily/date=2023-04-01
Processing date: 2023-05-01
Saved data to Parquet table at: datamart/bronze/lms_loan_daily/date=2023-05-01
Processing date: 2023-06-01
Saved data to Parquet table at: datamart/bronze/lms_loan_daily/date=2023-06-01
Processing date: 2023-07-01
Saved data to Parquet table at: datamart/bronze/lms_loan_daily/date=2023-07-01
Processing date: 2023-08-01
Saved data to Parquet table at: datamart/bronze/lms_loan_daily/date=2023-08-01
Processing date: 2023-09-01
Saved data to Parquet table at: datamart/bronze/lms_loan_daily/date=2023-09-01
Processing date: 2023-10-01
Saved data to Parquet table at: datam

## Build Silver Table

In [6]:
# bronze_directory = 'datamart/bronze/lms_loan_daily'
# silver_directory = 'datamart/silver/lms_loan_daily_cleaned'
# processor.process_bronze_to_silver(
#     bronze_path=bronze_directory,
#     silver_path=silver_directory,
#     dates=dates_str_lst,
#     source_type="lms_loan_daily"
# )

In [7]:
# bronze_directory = 'datamart/bronze/feature_clickstream'
# silver_directory = 'datamart/silver/feature_clickstream_cleaned'
# processor.process_bronze_to_silver(
#     bronze_path=bronze_directory,
#     silver_path=silver_directory,
#     dates=dates_str_lst,
#     source_type="feature_clickstream"
# )

In [8]:
bronze_directory = 'datamart/bronze/features_financials'
silver_directory = 'datamart/silver/features_financials_cleaned'
processor.process_bronze_to_silver(
    bronze_path=bronze_directory,
    silver_path=silver_directory,
    dates=dates_str_lst,
    source_type="features_financials"
)

2025-05-15 13:02:32,897 - INFO - Schema validation passed.
2025-05-15 13:02:33,604 - INFO - Initial DataFrame count after filtering for 2023-01-01: 530
2025-05-15 13:02:33,994 - INFO - Processed DataFrame count for 2023-01-01: 530
2025-05-15 13:02:33,995 - INFO - Processed DataFrame schema for 2023-01-01: StructType([StructField('Customer_ID', StringType(), True), StructField('Annual_Income', FloatType(), True), StructField('Monthly_Inhand_Salary', DoubleType(), True), StructField('Num_Bank_Accounts', IntegerType(), True), StructField('Num_Credit_Card', IntegerType(), True), StructField('Interest_Rate', IntegerType(), True), StructField('Num_of_Loan', IntegerType(), True), StructField('Delay_from_due_date', IntegerType(), True), StructField('Num_of_Delayed_Payment', FloatType(), True), StructField('Changed_Credit_Limit', StringType(), True), StructField('Num_Credit_Inquiries', IntegerType(), True), StructField('Credit_Mix', IntegerType(), True), StructField('Outstanding_Debt', StringTy

In [9]:
# bronze_directory = 'datamart/bronze/features_attributes'
# silver_directory = 'datamart/silver/features_attributes_cleaned'
# processor.process_bronze_to_silver(
#     bronze_path=bronze_directory,
#     silver_path=silver_directory,
#     dates=dates_str_lst,
#     source_type="features_attributes"
# )

## Build gold table for labels

In [10]:
# # create bronze datalake
# gold_label_store_directory = "datamart/gold/label_store/"

# if not os.path.exists(gold_label_store_directory):
#     os.makedirs(gold_label_store_directory)

In [11]:
# # run gold backfill
# for date_str in dates_str_lst:
#     utils.data_processing_gold_table.process_labels_gold_table(date_str, silver_loan_daily_directory, gold_label_store_directory, spark, dpd = 30, mob = 6)


In [12]:
# utils.data_processing_gold_table.process_labels_gold_table(date_str, silver_loan_daily_directory, gold_label_store_directory, spark, dpd = 30, mob = 6).dtypes


## inspect label store

In [13]:
# folder_path = gold_label_store_directory
# files_list = [folder_path+os.path.basename(f) for f in glob.glob(os.path.join(folder_path, '*'))]
# df = spark.read.option("header", "true").parquet(*files_list)
# print("row_count:",df.count())

# df.show()

In [14]:
# df.printSchema()

In [15]:
lms_df = spark.read.parquet('datamart/silver/lms_loan_daily_cleaned')
lms_df.show()

+--------------------+-----------+---------------+------+---------------+--------+-------+--------+-----------+-------+--------------------+-------------+
|             loan_id|Customer_ID|loan_start_date|tenure|installment_num|loan_amt|due_amt|paid_amt|overdue_amt|balance| _metadata_file_name|snapshot_date|
+--------------------+-----------+---------------+------+---------------+--------+-------+--------+-----------+-------+--------------------+-------------+
|CUS_0x5436_2024_0...| CUS_0x5436|     2024-06-01|    10|              3| 10000.0| 1000.0|  1000.0|        0.0| 7000.0|file:///app/data/...|   2024-09-01|
|CUS_0x6020_2024_0...| CUS_0x6020|     2024-03-01|    10|              6| 10000.0| 1000.0|  1000.0|        0.0| 4000.0|file:///app/data/...|   2024-09-01|
|CUS_0x675_2024_06_01|  CUS_0x675|     2024-06-01|    10|              3| 10000.0| 1000.0|  1000.0|        0.0| 7000.0|file:///app/data/...|   2024-09-01|
|CUS_0x676b_2023_1...| CUS_0x676b|     2023-11-01|    10|             

In [16]:
cs_df = spark.read.parquet('datamart/silver/feature_clickstream_cleaned')
cs_df.show()

+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----------+--------------------+-------------+
|fe_1|fe_2|fe_3|fe_4|fe_5|fe_6|fe_7|fe_8|fe_9|fe_10|fe_11|fe_12|fe_13|fe_14|fe_15|fe_16|fe_17|fe_18|fe_19|fe_20|Customer_ID| _metadata_file_name|snapshot_date|
+----+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----------+--------------------+-------------+
| 200| -36|  32| 180| -66| 134| 139|  -6| 151|  334|   73|   22|  145|   -1|  104|   49|  -24|  127|  -86|  162| CUS_0x3fd9|file:///app/data/...|   2023-02-01|
|  44| 111|  87| 100| 108| 173|  63|  58| 303|   57|  285|  202|   82|  196|  177|  111|  238|   60|  135|   63| CUS_0x527f|file:///app/data/...|   2023-02-01|
| 123|  76| 192|  17|  91| 224| 112| 136| -75|  -61|  131|   64|   95|  124|  -89|  -82|  162|  224|  -20|  208| CUS_0x8fb4|file:///app/data/...|   2023-02-01|
| 188|  46| 307|  67|  81| 200|  37|-103

In [17]:
fa_df = spark.read.parquet('datamart/silver/features_attributes_cleaned')
fa_df.show()

+-----------+--------------------+---+-----------+-------------+--------------------+-------------+
|Customer_ID|                Name|Age|        SSN|   Occupation| _metadata_file_name|snapshot_date|
+-----------+--------------------+---+-----------+-------------+--------------------+-------------+
| CUS_0x3db1|                Timz| 44|091-80-2935|   Journalist|file:///app/data/...|   2024-08-01|
|  CUS_0x552| Harry Papachristous| 48|837-55-2459|     Engineer|file:///app/data/...|   2024-08-01|
| CUS_0x5a04|   Catherine Bosleyt| 21|641-94-4207|     Engineer|file:///app/data/...|   2024-08-01|
| CUS_0x84be|Patrick Temple-Westj| 36|463-53-3692|Media_Manager|file:///app/data/...|   2024-08-01|
| CUS_0xae96|               Akiny| 43|414-33-7358|     Mechanic|file:///app/data/...|   2024-08-01|
| CUS_0x32b1|              Stevef| 45|517-36-8021|      Manager|file:///app/data/...|   2024-08-01|
| CUS_0x3e89|           Alistairx| 25|598-55-8096|Media_Manager|file:///app/data/...|   2024-08-01|


In [18]:
fa_df = spark.read.parquet('datamart/bronze/features_financials')
fa_df.printSchema()

root
 |-- Customer_ID: string (nullable = true)
 |-- Annual_Income: string (nullable = true)
 |-- Monthly_Inhand_Salary: double (nullable = true)
 |-- Num_Bank_Accounts: integer (nullable = true)
 |-- Num_Credit_Card: integer (nullable = true)
 |-- Interest_Rate: integer (nullable = true)
 |-- Num_of_Loan: string (nullable = true)
 |-- Type_of_Loan: string (nullable = true)
 |-- Delay_from_due_date: integer (nullable = true)
 |-- Num_of_Delayed_Payment: string (nullable = true)
 |-- Changed_Credit_Limit: string (nullable = true)
 |-- Num_Credit_Inquiries: integer (nullable = true)
 |-- Credit_Mix: string (nullable = true)
 |-- Outstanding_Debt: string (nullable = true)
 |-- Credit_Utilization_Ratio: double (nullable = true)
 |-- Credit_History_Age: string (nullable = true)
 |-- Payment_of_Min_Amount: string (nullable = true)
 |-- Total_EMI_per_month: double (nullable = true)
 |-- Amount_invested_monthly: string (nullable = true)
 |-- Payment_Behaviour: string (nullable = true)
 |-- Mon